In [5]:
# pip install BeautifulSoup4

In [6]:
from datetime import datetime


In [7]:
import gzip
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import time
import random
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
nm=gzip.open('../Other Source Data/IMDB/name.basics.tsv.gz','rb')
df_names = pd.read_csv(nm,sep='\t', low_memory=False)
df_names.head(10)


# # Index nconst	primaryName	birthYear	deathYear	primaryProfession	knownForTitles
# # 0	nm0000001	Fred Astaire	1899	1987	soundtrack,actor,miscellaneous	tt0050419,tt0031983,tt0072308,tt0053137
# # 1	nm0000002	Lauren Bacall	1924	2014	actress,soundtrack	tt0117057,tt0071877,tt0038355,tt0037382
# # 2	nm0000003	Brigitte Bardot	1934	\N	actress,soundtrack,music_department	tt0057345,tt0049189,tt0056404,tt0054452
# # 3	nm0000004	John Belushi	1949	1982	actor,soundtrack,writer	tt0072562,tt0077975,tt0078723,tt0080455
# # 4	nm0000005	Ingmar Bergman	1918	2007	writer,director,actor	tt0083922,tt0069467,tt0060827,tt0050986
# # 5	nm0000006	Ingrid Bergman	1915	1982	actress,soundtrack,producer	tt0077711,tt0038109,tt0036855,tt0034583
# # 6	nm0000007	Humphrey Bogart	1899	1957	actor,soundtrack,producer	tt0037382,tt0043265,tt0042593,tt0034583
# # 7	nm0000008	Marlon Brando	1924	2004	actor,soundtrack,director	tt0078788,tt0068646,tt0070849,tt0047296
# # 8	nm0000009	Richard Burton	1925	1984	actor,soundtrack,producer	tt0057877,tt0087803,tt0061184,tt0059749
# # 9	nm0000010	James Cagney	1899	1986	actor,soundtrack,director	tt0031867,tt0029870,tt0042041,tt0035575

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0031983,tt0072308,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0071877,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0077975,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0069467,tt0060827,tt0050986"
5,nm0000006,Ingrid Bergman,1915,1982,"actress,soundtrack,producer","tt0077711,tt0038109,tt0036855,tt0034583"
6,nm0000007,Humphrey Bogart,1899,1957,"actor,soundtrack,producer","tt0037382,tt0043265,tt0042593,tt0034583"
7,nm0000008,Marlon Brando,1924,2004,"actor,soundtrack,director","tt0078788,tt0068646,tt0070849,tt0047296"
8,nm0000009,Richard Burton,1925,1984,"actor,soundtrack,producer","tt0057877,tt0087803,tt0061184,tt0059749"
9,nm0000010,James Cagney,1899,1986,"actor,soundtrack,director","tt0031867,tt0029870,tt0042041,tt0035575"


In [9]:
# aka=gzip.open('IMDB/title.akas.tsv.gz','rb')
# df_akas = pd.read_csv(aka,sep='\t', low_memory=False)
# df_akas.head(10)


# # index titleId	ordering	title	region	language	types	attributes	isOriginalTitle
# # 0	tt0000001	1	Карменсіта	UA	\N	imdbDisplay	\N	0
# # 1	tt0000001	2	Carmencita	DE	\N	\N	literal title	0
# # 2	tt0000001	3	Carmencita - spanyol tánc	HU	\N	imdbDisplay	\N	0
# # 3	tt0000001	4	Καρμενσίτα	GR	\N	imdbDisplay	\N	0
# # 4	tt0000001	5	Карменсита	RU	\N	imdbDisplay	\N	0
# # 5	tt0000001	6	Carmencita	US	\N	imdbDisplay	\N	0
# # 6	tt0000001	7	Carmencita	\N	\N	original	\N	1
# # 7	tt0000001	8	カルメンチータ	JP	ja	imdbDisplay	\N	0
# # 8	tt0000002	1	Le clown et ses chiens	\N	\N	original	\N	1
# # 9	tt0000002	2	Le clown et ses chiens	FR	\N	imdbDisplay	\N	0

In [10]:
# since we are not considering anything other than full-length features, this may not be relevant
# ep=gzip.open('IMDB/title.episode.tsv.gz','rb')
# df_episodes = pd.read_csv(ep,sep='\t', low_memory=False)
# df_episodes.head()

# # Index tconst	parentTconst	seasonNumber	episodeNumber
# # 0	tt0020666	tt15180956	1	2
# # 1	tt0020829	tt15180956	1	1
# # 2	tt0021166	tt15180956	1	3
# # 3	tt0021612	tt15180956	2	2
# # 4	tt0021655	tt15180956	2	5

In [11]:
pr=gzip.open('../Other Source Data/IMDB/title.principals.tsv.gz','rb')
df_principals = pd.read_csv(pr,sep='\t', low_memory=False)
df_principals.head()

# # Index tconst	ordering	nconst	category	job	characters
# # 0	tt0000001	1	nm1588970	self	\N	["Self"]
# # 1	tt0000001	2	nm0005690	director	\N	\N
# # 2	tt0000001	3	nm0374658	cinematographer	director of photography	\N
# # 3	tt0000002	1	nm0721526	director	\N	\N
# # 4	tt0000002	2	nm1335271	composer	\N	\N

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [12]:
# rt=gzip.open('IMDB/title.ratings.tsv.gz','rb')
# df_ratings = pd.read_csv(rt,sep='\t', low_memory=False)
# df_ratings.head()
# # Index	tconst	averageRating	numVotes
# # 0	tt0000001	5.7	1868
# # 1	tt0000002	5.9	247
# # 2	tt0000003	6.5	1640
# # 3	tt0000004	5.8	159
# # 4	tt0000005	6.2	2463

# Get the Directors and Writers from IMDB using the tconst values

In [14]:
Disney_List = pd.read_csv('../Bens_Data/Disney_imdb_scrape_final.csv')

In [15]:
Disney_List.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439 entries, 0 to 438
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      439 non-null    int64  
 1   DFL_title       439 non-null    object 
 2   tconst          439 non-null    object 
 3   titleType       439 non-null    object 
 4   primaryTitle    439 non-null    object 
 5   originalTitle   439 non-null    object 
 6   startYear       439 non-null    int64  
 7   runtimeMinutes  439 non-null    int64  
 8   genres          439 non-null    object 
 9   lower_title     439 non-null    object 
 10  averageRating   439 non-null    float64
 11  numVotes        439 non-null    float64
 12  newurl          439 non-null    object 
 13  rating          439 non-null    float64
 14  votes           439 non-null    object 
 15  year            439 non-null    int64  
 16  mpaarating      439 non-null    object 
 17  runtime         439 non-null    obj

In [17]:
cr=gzip.open('../Other Source Data/IMDB/title.crew.tsv.gz','rb')
df_crews = pd.read_csv(cr,sep='\t', low_memory=False)
df_crews.tail(10)


# # index tconst	directors	writers
# # 8784762	tt9916838	nm5519375,nm5519454	nm6182221,nm1628284,nm2921377
# # 8784763	tt9916840	nm0996406	nm1482639,nm2586970
# # 8784764	tt9916842	nm5519454,nm5519375	nm6182221,nm1628284,nm2921377
# # 8784765	tt9916844	nm5519454,nm5519375	nm6182221,nm1628284,nm2921377
# # 8784766	tt9916846	nm5519375,nm5519454	nm6182221,nm1628284,nm2921377
# # 8784767	tt9916848	nm5519454,nm5519375	nm6182221,nm1628284,nm2921377
# # 8784768	tt9916850	nm5519454,nm5519375	nm6182221,nm1628284,nm2921377
# # 8784769	tt9916852	nm5519454,nm5519375	nm6182221,nm1628284,nm2921377
# # 8784770	tt9916856	nm10538645	nm6951431
# # 8784771	tt9916880	nm0996406	nm1482639,nm2586970

,tconst,directors,writers
8784762,tt9916838,"nm5519375,nm5519454","nm6182221,nm1628284,nm2921377"
8784763,tt9916840,nm0996406,"nm1482639,nm2586970"
8784764,tt9916842,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
8784765,tt9916844,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
8784766,tt9916846,"nm5519375,nm5519454","nm6182221,nm1628284,nm2921377"
8784767,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
8784768,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
8784769,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
8784770,tt9916856,nm10538645,nm6951431
8784771,tt9916880,nm0996406,"nm1482639,nm2586970"


In [18]:
Disney_Crews_combined = pd.merge(Disney_List ,                 # left df
                          df_crews,                  # right df
                          how="left",                 # left join
                          left_on='tconst',            # left column
                          right_on='tconst',    # right column
                          indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )

In [19]:
Disney_Crews_combined.head()

,Unnamed: 0,DFL_title,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,lower_title,...,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,directors,writers,_merge
0,0,101 Dalmatians,tt0115433,movie,101 Dalmatians,101 Dalmatians,1996,103,"Adventure,Comedy,Crime",101 dalmatians,...,110K,1996,G,1h 43m,"November 27, 1996 (United States)","$75,000,000 (estimated)","$320,689,294",nm0378893,"nm0807977,nm0000455",both
1,1,102 Dalmatians,tt0211181,movie,102 Dalmatians,102 Dalmatians,2000,100,"Adventure,Comedy,Family",102 dalmatians,...,37K,2000,G,1h 40m,"November 22, 2000 (United States)","$85,000,000 (estimated)","$183,611,771",nm0510674,"nm0807977,nm0118649,nm0716391,nm0879318,nm0925276",both
2,2,"20,000 Leagues Under the Sea",tt0046672,movie,"20,000 Leagues Under the Sea","20,000 Leagues Under the Sea",1954,127,"Adventure,Drama,Family",20000 leagues under the sea,...,33K,1954,G,2h 7m,"July 20, 1955 (United States)","$9,000,000 (estimated)",NaN,nm0281507,"nm0271641,nm0894523",both
3,3,A Bug's Life,tt0120623,movie,A Bug's Life,A Bug's Life,1998,95,"Adventure,Animation,Comedy",a bugs life,...,285K,1998,G,1h 35m,"November 25, 1998 (United States)","$120,000,000 (estimated)","$363,258,859","nm0005124,nm0004056","nm0005124,nm0004056,nm0710020,nm0568490,nm0789624",both
4,4,A Christmas Carol,tt1067106,movie,A Christmas Carol,A Christmas Carol,2009,96,"Adventure,Animation,Comedy",a christmas carol,...,113K,2009,PG,1h 36m,"November 6, 2009 (United States)","$200,000,000 (estimated)","$325,286,646",nm0000709,"nm0002042,nm0000709",both


In [20]:
Disney_Crews_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439 entries, 0 to 438
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Unnamed: 0      439 non-null    int64   
 1   DFL_title       439 non-null    object  
 2   tconst          439 non-null    object  
 3   titleType       439 non-null    object  
 4   primaryTitle    439 non-null    object  
 5   originalTitle   439 non-null    object  
 6   startYear       439 non-null    int64   
 7   runtimeMinutes  439 non-null    int64   
 8   genres          439 non-null    object  
 9   lower_title     439 non-null    object  
 10  averageRating   439 non-null    float64 
 11  numVotes        439 non-null    float64 
 12  newurl          439 non-null    object  
 13  rating          439 non-null    float64 
 14  votes           439 non-null    object  
 15  year            439 non-null    int64   
 16  mpaarating      439 non-null    object  
 17  runtime         

In [21]:
# There are 314 unique values, but we know some of those values contain multiple directors. 
# When we use CountVectorizer to get the unique list of directors, 
# we should ens up with more than 314 unique directors

len(Disney_Crews_combined['directors'].unique())

314

In [22]:
# For each film,
#     for each dirctor on the film
#         get the name of each director
#         get the total number of films that director worked on PRIOR to this one
#         get the budget of each film that director worked on PRIOR to this one
#         get the revenue of each film that director worked on PRIOR to this one
#         get the number of DISNEY films that director worked on PRIOR to this one
#         get the budget of each DISNEY film that director worked on PRIOR to this one
#         get the revenue of each DISNEY film that director worked on PRIOR to this one
#         get the total number of films that director DIRECTED PRIOR to this one
#         get the budget of each film that director DIRECTED PRIOR to this one
#         get the revenue of each film that director DIRECTED PRIOR to this one
#         get the number of DISNEY films that director DIRECTED PRIOR to this one
#         get the budget of each DISNEY film that director DIRECTED PRIOR to this one
#         get the revenue of each DISNEY film that director DIRECTED PRIOR to this one
#    If there is more than one director,
#        get the AVERAGE for each of the above
#        get the SUM for each of the above

In [23]:
# 1 Get a list of unique Disney films / directors combos (film, tconst, director name, nconst, "director") 
# 2 Get a list of unique Disney films / writers combos (film, tconst, writer name, nconst, "writer") 
# 3 Get a list of the unique Directors
# 4 Get a list of the unique Writers
# ( SOME NAMES MIGHT APPEAR ON BOTH LISTS BUT EACH NAME SHOULD ONLY APPEAR ON EACH LIST ONE TIME)
# 5 Create a list of each film that each Director directed (director name, nconst, "director",film, tconst, rls date, budget, revenue )
# 6 Create a list of each film that wach Writer wrote (writer name, nconst, "writer",film, tconst, rls date, budget, revenue )
# 7 Determine if each film is a Disney Movie or Not Disney Movie
# 8 Either average or meadian the missing budget numbers
# 9 Either average or meadian the missing revenue numbers
# 10 Calculate ROI for each list

#### Create a list of directors from Disney Films

In [24]:
cv = CountVectorizer()
cvec = cv.fit(Disney_Crews_combined['directors'])
csr = cvec.transform(Disney_Crews_combined['directors'])
disney_directors = pd.DataFrame(csr.todense(), columns=cvec.get_feature_names())
disney_directors = disney_directors.T
disney_directors.reset_index(inplace=True)
disney_directors.rename(columns={"index": "nconst"},inplace=True)
disney_directors = disney_directors[['nconst']]
disney_directors['director'] = 'director'


disney_directors = pd.merge(disney_directors ,                 # left df
                          df_names[['nconst', 'primaryName', 'birthYear']],                  # right df
                          how="left",                 # left join
                          left_on='nconst',            # left column
                          right_on='nconst',    # right column
                          indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )

disney_directors['_merge'].value_counts()

both          349
left_only       0
right_only      0
Name: _merge, dtype: int64

In [25]:
disney_directors.drop(columns='_merge',inplace = True)
disney_directors.head()

,nconst,director,primaryName,birthYear
0,nm0000110,director,Kenneth Branagh,1960
1,nm0000116,director,James Cameron,1954
2,nm0000165,director,Ron Howard,1954
3,nm0000186,director,David Lynch,1946
4,nm0000200,director,Bill Paxton,1955


#### Create a list of all films the Disney Directors have worked on, Disney or Otherwise

In [26]:
df_directors = df_principals[df_principals['category']=='director']

In [27]:
disney_directors_history = pd.merge(disney_directors ,                 # left df
                          df_directors[['tconst', 'nconst', 'category']],                  # right df
                          how="left",                 # left join
                          left_on='nconst',            # left column
                          right_on='nconst',    # right column
                          #indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )

In [28]:
disney_directors_history.head()

,nconst,director,primaryName,birthYear,tconst,category
0,nm0000110,director,Kenneth Branagh,1960,tt0105506,director
1,nm0000110,director,Kenneth Branagh,1960,tt0113403,director
2,nm0000110,director,Kenneth Branagh,1960,tt0357865,director
3,nm0000110,director,Kenneth Branagh,1960,tt0450972,director
4,nm0000110,director,Kenneth Branagh,1960,tt0475331,director


#### Create a list of writers from Disney Films

In [29]:
cvec = cv.fit(Disney_Crews_combined['writers'])
csr = cvec.transform(Disney_Crews_combined['writers'])
disney_writers = pd.DataFrame(csr.todense(), columns=cvec.get_feature_names())
disney_writers = disney_writers.T
disney_writers.reset_index(inplace=True)
disney_writers.rename(columns={"index": "nconst"},inplace=True)
disney_writers = disney_writers[['nconst']]
disney_writers['writer'] = 'writer'

disney_writers = pd.merge(disney_writers ,                 # left df
                          df_names[['nconst', 'primaryName', 'birthYear']],                  # right df
                          how="left",                 # left join
                          left_on='nconst',            # left column
                          right_on='nconst',    # right column
                          indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )
disney_writers['_merge'].value_counts()

both          1093
left_only        0
right_only       0
Name: _merge, dtype: int64

In [30]:
disney_writers.drop(columns='_merge',inplace = True)
disney_writers.head()

,nconst,writer,primaryName,birthYear
0,nm0000184,writer,George Lucas,1944
1,nm0000318,writer,Tim Burton,1958
2,nm0000370,writer,Walt Disney,1901
3,nm0000436,writer,Curtis Hanson,1945
4,nm0000455,writer,John Hughes,1950


#### Create a list of all films the Disney Writers have worked on, Disney or Otherwise

In [31]:
df_writers = df_principals[df_principals['category']=='writer']

In [32]:
disney_writers_history = pd.merge(disney_writers ,                 # left df
                          df_writers[['tconst', 'nconst', 'category']],                  # right df
                          how="left",                 # left join
                          left_on='nconst',            # left column
                          right_on='nconst',    # right column
                          #indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )

In [33]:
disney_writers_history.head()

,nconst,writer,primaryName,birthYear,tconst,category
0,nm0000184,writer,George Lucas,1944,tt0079576,writer
1,nm0000184,writer,George Lucas,1944,tt0080684,writer
2,nm0000184,writer,George Lucas,1944,tt0082971,writer
3,nm0000184,writer,George Lucas,1944,tt0086190,writer
4,nm0000184,writer,George Lucas,1944,tt0087225,writer


## For our Directors History and Writers History, we need to get the realse date for each film they worked on, as well as the budget and revenue.
### The title and release date can come from data already provided by IMDB.
### The budget and revenue numbers will need to be scraped from IMDB.

In [34]:
disney_directors_history.head(3)

,nconst,director,primaryName,birthYear,tconst,category
0,nm0000110,director,Kenneth Branagh,1960,tt0105506,director
1,nm0000110,director,Kenneth Branagh,1960,tt0113403,director
2,nm0000110,director,Kenneth Branagh,1960,tt0357865,director


In [35]:
disney_writers_history.head(3)

,nconst,writer,primaryName,birthYear,tconst,category
0,nm0000184,writer,George Lucas,1944,tt0079576,writer
1,nm0000184,writer,George Lucas,1944,tt0080684,writer
2,nm0000184,writer,George Lucas,1944,tt0082971,writer


### To make things a little easier to manage, we're going to get a list of unique tconst values from Disney Directors and Disney Writers, use the combined list to gather all of the unformation, then marry the film information to the Director and Writer.

In [36]:
len(disney_directors_history['tconst'].unique()), len(disney_writers_history['tconst'].unique())

(18366, 51988)

In [37]:
comb = np.concatenate((disney_directors_history['tconst'].unique(), disney_writers_history['tconst'].unique()), axis=0)

In [38]:
df_comb = pd.DataFrame(comb, columns = ['tconst'])
df_comb = df_comb['tconst'].unique()
df_comb = pd.DataFrame(df_comb, columns = ['tconst'])
df_comb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67296 entries, 0 to 67295
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tconst  67295 non-null  object
dtypes: object(1)
memory usage: 525.9+ KB


In [40]:
bs=gzip.open('../Other Source Data/IMDB/title.basics.tsv.gz','rb')
df_basics = pd.read_csv(bs,sep='\t', low_memory=False)

In [41]:
df_basics.shape

(8784772, 9)

In [42]:
df_basics_movies= df_basics[df_basics['titleType']=='movie']
df_basics_movies.shape

(605156, 9)

In [43]:
df_basics_movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama
625,tt0000630,movie,Hamlet,Amleto,0,1908,\N,\N,Drama


In [44]:
directors_writers_combined_history = pd.merge(df_comb ,                 # left df
                          df_basics_movies[['tconst', 'titleType', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres']],                  # right df
                          how="left",                 # left join
                          left_on='tconst',            # left column
                          right_on='tconst',    # right column
                          #indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )
directors_writers_combined_history.shape

(67296, 6)

### All of the writers and directors, combined, have worked on 7837 unique projects

In [45]:
directors_writers_combined_history = directors_writers_combined_history[directors_writers_combined_history['titleType'] == 'movie']
directors_writers_combined_history.shape

(7837, 6)

In [47]:
directors_writers_combined_history.to_csv('../Bens_Data/directors_writers_combined_history.csv')

# Now it's time to scrape the data we need from IMDB

There were a few false starts in the section below.  The scrap takes a long time to run and acpatures a lot of data.  We ran in to timing issues and mempry issues.  Even running the script Google required a couple of restarts to gather all of the data.  Given more time and resources, we would have tried to make this code more efficient and written it as a python script that could run on a VM, rather than from a jupyter notebook.

In [47]:
# pip install lxml

In [48]:
# Target web page:
url = "https://www.imdb.com/title/"
# Establishing the connection to the web page:
response = requests.get(url)
# response.text[:50]
soup = BeautifulSoup(response.text, 'lxml')

imdb_errors = pd.DataFrame(columns=['tconst'])
# imdb_soup = pd.DataFrame(columns=['tconst','newurl','soup_text'])
# imdb_scrape = pd.DataFrame(columns=['tconst','newurl','mpaarating','rlsdt','budget','wordlwide' ])


In [5]:
imdb_soup = pd.read_csv('imdb_scrape_director_writer_hist_SOUP.csv')
imdb_scrape = pd.read_csv('imdb_scrape_director_writer_hist.csv')

In [49]:
imdb_soup.shape, imdb_scrape.shape

((4557, 3), (5898, 6))

In [50]:
imdb_soup.drop(columns='Unnamed: 0',inplace=True)
imdb_scrape.drop(columns='Unnamed: 0',inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [51]:
imdb_scrape.head()

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide
0,tt0113403,https://www.imdb.com/title/tt0113403,R,"February 16, 1996 (United States)",error,"$469,571"
1,tt0450972,https://www.imdb.com/title/tt0450972,PG,"September 21, 2007 (United Kingdom)",error,"$563,162"
2,tt0475331,https://www.imdb.com/title/tt0475331,error,"December 13, 2006 (France)","$27,000,000 (estimated)","$2,000,853"
3,tt0800369,https://www.imdb.com/title/tt0800369,PG-13,"May 6, 2011 (United States)","$150,000,000 (estimated)","$449,326,618"
4,tt11229040,https://www.imdb.com/title/tt11229040,error,error,error,error


In [ ]:
for const in directors_writers_combined_history['tconst']:
    found1 = imdb_scrape[imdb_scrape['tconst'].str.contains(const)]
    if found1.count().sum() == 0:
        print(const)
        print(found1.count().sum())

In [52]:
counter = 1
for const in directors_writers_combined_history['tconst']:
        
        found1 = imdb_scrape[imdb_scrape['tconst'].str.contains(const)]
        if found1.count().sum() == 0:
        
            t= random.uniform(.4, 1)
            time.sleep(t)
            newurl = url + const
            response = requests.get(newurl)
            soup = BeautifulSoup(response.text, 'lxml')


            print(counter)
            counter += 1
            print(newurl)

            try:
                metaul = soup.find_all('ul', {'data-testid' : 'hero-title-block__metadata'})
                mpaarating = metaul[0].find_all('li')[1].find_all('span')[0].get_text()
                print(mpaarating) 
            except:
                print('mpaarating error')
                mpaarating='error'
                imdb_errors.loc[len(imdb_errors.index)] = [const]
                # imdb__errors_soup.loc[len(imdb_soup.index)] = [const, newurl, soup]

            try:
                rlsdt = soup.find_all('li', {'data-testid' : 'title-details-releasedate'})[0].find_all('a')[1].get_text()
                print(rlsdt) 
            except:
                print('rlsdt error')
                rlsdt='error'
                imdb_errors.loc[len(imdb_errors.index)] = [const]
                # imdb__errors_soup.loc[len(imdb_soup.index)] = [const, newurl, soup]

            try:
                budget = soup.find_all('li', {'data-testid' : 'title-boxoffice-budget'})[0].find_all('li')[0].get_text()
                print(budget) 
            except:
                print('budget error')
                budget='error'
                imdb_errors.loc[len(imdb_errors.index)] = [const]
                # imdb__errors_soup.loc[len(imdb_soup.index)] = [const, newurl, soup]

            try:
                worldwide = soup.find_all('li', {'data-testid' : 'title-boxoffice-cumulativeworldwidegross'})[0].find_all('li')[0].get_text()
                print(worldwide) 
            except:
                print('worldwide error')
                worldwide='error'
                imdb_errors.loc[len(imdb_errors.index)] = [const]
                # imdb__errors_soup.loc[len(imdb_soup.index)] = [const, newurl, soup]



            imdb_scrape.loc[len(imdb_scrape.index)] = [const, newurl, mpaarating ,rlsdt ,budget ,worldwide]
            imdb_soup.loc[len(imdb_soup.index)] = [const, newurl, soup]



            if counter % 100 == 0:
                imdb_scrape.to_csv('imdb_scrape_director_writer_hist.csv')
                imdb_soup.to_csv('imdb_scrape_director_writer_hist_SOUP.csv')
                now = datetime.now()
                print(f'***************************  TO CSV : {counter} rows ***************************')
                print(now.strftime("%H:%M:%S"))
                print(f'********************************************************************************')

1
https://www.imdb.com/title/tt0003765
mpaarating error
November 23, 1914 (United States)
budget error
worldwide error
2
https://www.imdb.com/title/tt0004175
mpaarating error
July 1914 (United States)
budget error
worldwide error
3
https://www.imdb.com/title/tt0004307
mpaarating error
August 16, 1914 (United States)
budget error
worldwide error
4
https://www.imdb.com/title/tt0004415
mpaarating error
September 3, 1914 (United States)
budget error
worldwide error
5
https://www.imdb.com/title/tt0004757
mpaarating error
June 1914 (United States)
budget error
worldwide error
6
https://www.imdb.com/title/tt0008979
mpaarating error
rlsdt error
budget error
worldwide error
7
https://www.imdb.com/title/tt0011039
mpaarating error
May 1920 (United States)
budget error
worldwide error
8
https://www.imdb.com/title/tt0011489
mpaarating error
October 1920 (United States)
budget error
worldwide error
9
https://www.imdb.com/title/tt0011672
mpaarating error
May 16, 1920 (United States)
budget error
worl

In [ ]:
imdb_scrape.head()

In [ ]:
22:02

In [ ]:
7:22:02

In [53]:
imdb_scrape.to_csv('imdb_scrape_director_writer_hist.csv')
imdb_soup.to_csv('imdb_scrape_director_writer_hist_SOUP.csv')